In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [1]:
import pandas as pd

In [ ]:
#数据加载
def get_data(file_name):
    result = []
    chunk_index = 0
    for df in pd.read_csv(open(file_name,'r'),chunksize =1000000):
        result.append(df)
        print('chunk',chunk_index)
        chunk_index += 1
    result = pd.concat(result,ignore_index=True,axis=0)
    return result


In [ ]:
#获取全量数据
'''
train = get_data('./data/data100940/security_train.csv')

import pickle
with open('./train.pkl','wb') as f:
    pickle.dump(train,f)

import gc
del train
gc.collect()

test = get_data('./data/data100940/security_test.csv')

with open('./test.pkl','wb') as f:
    pickle.dump(test,f)

del test
gc.collect()
'''

In [ ]:
'''
#用pickle读取，不会内存意出
import pickle
with open('./train.pkl','rb') as f:
    train = pickle.load(f)

with open('./test.pkl','rb') as f:
    test = pickle.load(f)
'''

In [ ]:
#对api字段进行LabelEncoder
train['api'].describe()

count                   89806693
unique                       295
top       LdrGetProcedureAddress
freq                    10704305
Name: api, dtype: object

In [ ]:
'''
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
'''

In [ ]:
'''
#将训练集和测试机合并
df_all = pd.concat([train,test])
df_all['api'] = le.fit_transform(df_all['api'])
df_all[['api']]
'''

,api
0,135
1,134
2,134
3,134
4,134
...,...
79288370,266
79288371,266
79288372,152
79288373,281


In [ ]:
'''
#提取train['api']
train['api'] = df_all[df_all['label'].notnull()]['api']
test['api'] = df_all[df_all['label'].isnull()]['api']
'''


In [ ]:
'''
import gc
del df_all
gc.collect()
'''


113

In [ ]:
#构造新特征(基于fileID的聚合统计)
def get_features(df):
    #按照file_id分组，提取统计特征
    df_file = df.groupby('file_id')
    #df1为最终结果
    if 'label' in df.columns: #训练集
        df1 = df.drop_duplicates(subset=['file_id','label'],keep='first')
    else:
        df1 = df.drop_duplicates(subset=['file_id'],keep='first')
    df1 = df1.sort_values('file_id')
    #提取多个统计特征
    features = ['api','tid','index']
    for f in features:
        #针对file_id构造不同特征，一个file_id只有一行数据
        df1[f + '_count'] = df_file[f].count().values
        df1[f + '_nunique'] = df_file[f].nunique().values
        df1[f + '_min'] = df_file[f].min().values
        df1[f + '_max'] = df_file[f].max().values
        df1[f + '_mean'] = df_file[f].mean().values
        df1[f + '_median'] = df_file[f].median().values
        df1[f + '_std'] = df_file[f].std().values
        df1[f + '_ptp'] = df1[f + '_max'] - df1[f + '_min']

    return df1


In [ ]:
'''
df_train = get_features(train)
df_test = get_features(test)
df_train.to_pickle('./df_train.pkl')
df_test.to_pickle('./df_test.pkl')
'''

In [2]:
import pickle
with open('./df_train.pkl','rb') as file:
    df_train = pickle.load(file)
with open('./df_test.pkl','rb') as file:
    df_test = pickle.load(file)

In [3]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(
            num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='multiclass',
            max_depth=-1, learning_rate=0.005, min_child_samples=3, random_state=2021,
            n_estimators=2000, subsample=1, colsample_bytree=1)
clf.fit(df_train.drop('label',axis=1),df_train['label'])

LGBMClassifier(colsample_bytree=1, learning_rate=0.005, min_child_samples=3,
               n_estimators=2000, objective='multiclass', random_state=2021,
               reg_alpha=0.25, reg_lambda=0.25, subsample=1)

In [ ]:
import xgboost as xgb
model_xgb = xgb.XGBClassifier(
            max_depth=9, learning_rate=0.005, n_estimators=2000, 
            objective='multi:softprob', tree_method='gpu_hist', 
            subsample=0.8, colsample_bytree=0.8, 
            min_child_samples=3, eval_metric='logloss', reg_lambda=0.5)


In [4]:
result = clf.predict_proba(df_test)
result
result.shape

(12955, 8)

In [ ]:
result

array([7, 0, 0, ..., 5, 5, 5])

In [7]:
result = pd.DataFrame(result,columns=['prob0', 'prob1', 'prob2', 'prob3', 'prob4', 'prob5', 'prob6', 'prob7'])
result['file_id'] = df_test['file_id'].values
result

,prob0,prob1,prob2,prob3,prob4,prob5,prob6,prob7,file_id
0,0.012991,0.002973,0.116246,0.112650,0.000771,0.018385,0.008723,0.727260,1
1,0.875510,0.000939,0.005767,0.006461,0.000534,0.053984,0.018161,0.038644,2
2,0.998085,0.000038,0.000218,0.000072,0.000013,0.000740,0.000151,0.000684,3
3,0.045739,0.001098,0.004962,0.198713,0.001311,0.132568,0.026041,0.589569,4
4,0.989740,0.000035,0.001995,0.000887,0.000016,0.005407,0.000468,0.001452,5
...,...,...,...,...,...,...,...,...,...
12950,0.833071,0.003057,0.014412,0.003555,0.000654,0.034868,0.003584,0.106798,12951
12951,0.946363,0.000961,0.002724,0.002242,0.013856,0.023608,0.002090,0.008157,12952
12952,0.000735,0.000256,0.001295,0.000469,0.000031,0.973190,0.017754,0.006271,12953
12953,0.001971,0.000021,0.000765,0.000369,0.000025,0.994096,0.000372,0.002380,12954


In [9]:
columns =['file_id','prob0', 'prob1', 'prob2', 'prob3', 'prob4', 'prob5', 'prob6', 'prob7']
result.to_csv('./baseline_lgb_2000.csv',index=False,columns=columns)